In [1]:
import time
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

from harmonic import HarmonicUpdater

#### initializing snapshot

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=9520)
sim.create_state_from_gsd(filename='./DATA/lattice.gsd')

#### initializing integrator

In [3]:
alpha_init=1.0
family323 = coxeter.families.Family323Plus()
particle = family323.get_shape(a=alpha_init, c=0.2*alpha_init+0.8)
verts = particle.vertices/particle.volume**(1/3)

mc = hoomd.hpmc.integrate.ConvexPolyhedron()
mc.shape["A"] = dict(vertices=verts)

#### harmonic constraint

In [4]:
target = gsd.hoomd.open('./DATA/lattice.gsd', 'r')[-1]
pos = target.particles.position
ori = target.particles.orientation
box = target.configuration.box

k_init=20
k_rot_init=10

harmonic = hoomd.hpmc.external.field.Harmonic(reference_positions=pos,
                                              reference_orientations=ori,
                                              k_translational=k_init,
                                              k_rotational=k_rot_init,
                                              symmetries=ori)
mc.external_potential=harmonic

#### harmonic updater

In [5]:
harmonicupdater = hoomd.update.CustomUpdater(action=HarmonicUpdater(ref_pos=pos, 
                                                                    init_size=box[0]), trigger=1)

#### quick compress

In [6]:
final_volume_fraction = 0.57

initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)
V_particle = 1
final_box.volume = sim.state.N_particles * V_particle / final_volume_fraction
compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(1000),
                                           target_box=final_box)

periodic = hoomd.trigger.Periodic(10)
tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                             target=0.2,
                                             trigger=periodic,
                                             max_translation_move=0.2,
                                             max_rotation_move=0.2)

#### write

In [7]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

gsd_writer = hoomd.write.GSD(filename='./DATA/compress.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             mode='wb',
                             filter=hoomd.filter.All())
gsd_writer.logger=logger

#### attaching operations

In [8]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = mc
sim.operations.updaters.append(harmonicupdater)
sim.operations.updaters.append(compress)
sim.operations.tuners.append(tune)

#### run

In [9]:
while not compress.complete and sim.timestep < 1e6:
    sim.run(1000)

#### check compress

In [10]:
if not compress.complete:
    raise RuntimeError("Compression failed to complete")

In [11]:
mc.overlaps

0

In [12]:
gsd_writer.flush()